In [34]:
from flask import Flask, render_template, request
from textblob import TextBlob

In [35]:
#! usr/bin/env python3
import praw
import pandas as pd
import datetime as dt

In [36]:
import tweepy
from textblob import TextBlob
import csv
import re
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

In [37]:
reddit = praw.Reddit(client_id='HzXasZS7KlVL0w',client_secret='nFCTM7PCsPktAE7ibjponTKqtfw',user_agent='sentiment_analyser',username='nikhilmlal',password='Anbrothers4reddit')

In [38]:
c_key = 'cRjT9hq0hpoHjpCYM9c8XGTQe'
c_secret = '6he7tK6KH4A1oqVqiu3djCgpu1dtCHXijftKoVfIkyGdY2vUWi'
a_key = '740485466382868480-P9COIlo5B3ddDlndomlmWCyd1TvDRW1'
a_secret = 'gUbKMd2tuRJFugGwGZP77gUkxSIHQQoWlcJPiI4jAnk0k'

In [39]:
auth=tweepy.OAuthHandler(c_key,c_secret)
auth.set_access_token(a_key,a_secret)
api=tweepy.API(auth)

In [40]:
unwanted_words=['@','RT',':','https','http']
symbols=['@','#']
data=[]
score = 0

In [41]:
app = Flask(__name__)

In [42]:
@app.route('/')
def index():
    return render_template('test.html')

In [43]:
os.remove("reddit_analysis.csv")
os.remove("twitter_analysis.csv")

In [44]:
@app.route('/hello', methods=['POST'])
def hello():
    keyword = request.form['keyword']
    data1 = []
    data2 = []
    # Reddit Analysis
    analysis= TextBlob(keyword)
    subreddit = reddit.subreddit('all')
    results = subreddit.search(keyword,limit=15)
    result_posts = []
    
    for submission in subreddit.search(keyword,limit=20):
        result_posts.append(submission.title)
    
    score1 = 0
    for post in result_posts :
        text1 = post
        textWords1 = text1.split()
        #print (textWords)
        cleanedText = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT)", " ", text1).split())
        #print (cleanedText1)
        #print (TextBlob(cleanedTweet).tags)
        analysis1 = TextBlob(cleanedText)
        score1 = score1 + analysis1.sentiment.polarity
        #print (analysis1.sentiment)
        #print ("\n")
        polarity1 = 'Positive'
        if(analysis1.sentiment.polarity < 0):
            polarity1 = 'Negative'
        if(0<=analysis1.sentiment.polarity <=0.2):
            polarity1 = 'Neutral'
        #print (polarity)
        dic1={}
        dic1['Sentiment']=polarity1
        dic1['Post']=cleanedText
        data1.append(dic1)
    
    df1=pd.DataFrame(data1)
    df1.to_csv('reddit_analysis.csv')
    file1 = pd.read_csv('reddit_analysis.csv')
    x1 = file1["Sentiment"]
    plt.hist(x1,color=["lightgreen"],align='left')
    count1 = [0,0,0]
    for i in x1:
        if(i=="Positive"):
            count1[0]+=1
        elif(i=="Negative"):
            count1[1]+=1
        else:
            count1[2]+=1
            
            
    # Twitter Analysis
    
    pubic_tweets=api.search(keyword)
    score2 = 0
    for tweet in pubic_tweets:
        text2 = tweet.text
        textWords2=text2.split()
        #print (textWords)
        cleanedTweet=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT)", " ", text2).split())
        print (cleanedTweet)
        #print (TextBlob(cleanedTweet).tags)
        analysis2= TextBlob(cleanedTweet)
        score2 = score2 + analysis2.sentiment.polarity
        #print (analysis.sentiment)
        #print ("\n")
        polarity2 = 'Positive'
        if(analysis2.sentiment.polarity < 0):
            polarity2 = 'Negative'
        if(0<=analysis2.sentiment.polarity <=0.2):
            polarity2 = 'Neutral'
        #print (polarity)
        dic2={}
        dic2['Sentiment']=polarity2
        dic2['Tweet']=cleanedTweet
        data2.append(dic2)
        
        df2=pd.DataFrame(data2)
        df2.to_csv('twitter_analysis.csv')
        file2 = pd.read_csv('twitter_analysis.csv')
        x2 = file2["Sentiment"]
        plt.hist(x2,color=["lightblue"],align='right')
        count2 = [0,0,0]
        for i in x2:
            if(i=="Positive"):
                count2[0]+=1
            elif(i=="Negative"):
                count2[1]+=1
            else:
                count2[2]+=1
    
    return '<b>Twitter Data</b><br/>Sentiment score :<b> %s</b><br/>Positive :%s &#37;<br/>Negative :%s &#37;<br/>Neutral :%s &#37;<br/> <b>Reddit Data</b><br/>Sentiment score :<b> %s</b> <br/>Positive :%s &#37;<br/>Negative :%s &#37;<br/>Neutral :%s &#37;<br/> <a href="/">Back Home</a> ' % (score2, (count2[0]/len(x2))*100,(count2[1]/len(x2))*100,(count2[2]/len(x2))*100, score1,(count1[0]/len(x1))*100,(count1[1]/len(x1))*100,(count1[2]/len(x1))*100)

In [ ]:
import webbrowser
if __name__ == '__main__':
    a_website = "http://0.0.0.0:3000/"
    webbrowser.open_new_tab(a_website)
    app.run(host = '0.0.0.0', port = 3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2019 11:15:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2019 11:15:21] "GET / HTTP/1.1" 200 -
